#### **Loading Spam Classification Model**

Dataset used : <link>https://archive.ics.uci.edu/dataset/228/sms+spam+collection<link>

In [ ]:
import urllib.request
import ssl
import zipfile
import os
from pathfile import Path

url = 'https://archive.ics.uci.edu/dataset/228/sms+spam+collection'
zip_path = 'sms_spam_collection.zip'
extracted_path = 'sms_spam_collection'
data_file_path = Path(extracted_path) / 'SMSSpamCollection.tsv'


def download_and_unzip_spam_data(url, zip_path, extracted_path, data_file_path):
    if data_file_path.exists():
        print(f"{data_file_path}, already exists. Skipping download and extraction")
        return 
    
    # Creating unverfied SSL context
    ssl_context = ssl._create_unverified_context()

    # Download the file 
    with urllib.request.urlopen(url, context=ssl_context) as response:
        with open(zip_path, "wb") as f:
            f.write(response.read())

    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extracted_path)

    # Add .tsv file extension
    original_file_path = Path(extracted_path) / "SMSSpamCollection"
    os.rename(original_file_path, data_file_path)
    print(f"File downloaded and saved as : {data_file_path}")
        

